In [1]:
import lettuce as lt
import matplotlib.pyplot as plt
import numpy as np
import torch
import os
import csv
from argparse import ArgumentParser, ArgumentDefaultsHelpFormatter


In [5]:
parser = ArgumentParser(formatter_class=ArgumentDefaultsHelpFormatter)
parser.add_argument("--vtkdir", type=str, help="name base of vtk files - they are usually in a vtk-subdirectory!", default="./output/")
parser.add_argument("--resolution", type=int, default=16)
parser.add_argument("--nout", type=int, default=10)
parser.add_argument("--nvtk", type=int, default=200)
parser.add_argument("--tmax", type=int, default=10)
args, unknown = parser.parse_known_args()
args = vars(args)
#args = vars(parser.parse_args())
basedir = args["vtkdir"]
res = args["resolution"]
nout = args["nout"]
nvtk = args["nvtk"]
tmax = args["tmax"]
#print(args)
#print(unknown)

{'vtkdir': './output/', 'resolution': 16, 'nout': 10, 'nvtk': 200, 'tmax': 10}
['-f', '/home/ben/.local/share/jupyter/runtime/kernel-397984fc-9601-4f7b-a032-7e0b1d3edcc9.json']


In [3]:
lattice = lt.Lattice(lt.D3Q19, device = "cpu", dtype=torch.float64)
flow = lt.SuperReducedTaylorGreenVortex3D(resolution=res, reynolds_number=800, mach_number=0.05, lattice=lattice)
print(flow.units.convert_time_to_lu(tmax))
collision = lt.BGKCollision(lattice, tau=flow.units.relaxation_parameter_lu)
print(flow.units.relaxation_parameter_lu)
streaming = lt.StandardStreaming(lattice)
simulation = lt.SimulationReducedTGV(flow=flow, lattice=lattice, collision=collision, streaming=streaming)

RuntimeError: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero.

In [3]:
Energy = lt.IncompressibleKineticEnergy(lattice, flow)
Dissipation_TGV=lt.Dissipation_TGV(lattice, flow)
reporter = lt.ObservableReporter(Energy, interval=nout, out=None)
#Enstrophy=lt.Enstrophy(lattice,flow)
reporter2= lt.ObservableReporter(Dissipation_TGV, interval=nout, out=None)
simulation.reporters.append(reporter)
simulation.reporters.append(reporter2)
simulation.reporters.append(lt.VTKReporter(lattice, flow, interval=nvtk, filename_base=basedir+"out"))

steps     time     IncompressibleKineticEnergy
steps     time     Dissipation_TGV


In [4]:
simulation.initialize_f_neq()
steps = int(flow.units.convert_time_to_lu(tmax))
mlups = simulation.step(num_steps=steps)
print("Performance in MLUPS:", mlups)
#rho_start,u_start,f_start=simulation.startwerte()

Performance in MLUPS: 12.290149537758225
rho_start
tensor([[[[1.0003, 1.0003, 1.0003,  ..., 1.0009, 1.0009, 1.0009],
          [1.0003, 1.0003, 1.0003,  ..., 1.0009, 1.0009, 1.0009],
          [1.0003, 1.0003, 1.0003,  ..., 1.0009, 1.0009, 1.0009],
          ...,
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]],

         [[1.0003, 1.0003, 1.0003,  ..., 1.0009, 1.0009, 1.0009],
          [1.0003, 1.0003, 1.0003,  ..., 1.0009, 1.0009, 1.0009],
          [1.0003, 1.0003, 1.0003,  ..., 1.0009, 1.0009, 1.0009],
          ...,
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]],

         [[1.0003, 1.0003, 1.0003,  ..., 1.0009, 1.0009, 1.0009],
          [1.0003, 1.0003, 1.0003,  ..., 1.0009, 1.0009, 

In [5]:
energy = np.array(simulation.reporters[0].out)
print(energy.shape)

#save_directory = outputdir
#file_name = "FullTGV.pkl"
#file_path = os.path.join(save_directory, file_name)
#simulation.save_checkpoint(file_path)

csv_file_path = 'Brachet/Brachet_1983_tgv3d_enstrophy_Re800.csv'

# Daten aus der CSV-Datei lesen
csv_data = []
FullTGV_DisEns=[]
FullTGV_DisEkin=[]
FullTGV_DisSij=[]
with open(csv_file_path, 'r') as file:
    csv_reader = csv.reader(file, delimiter=';')
    for row in csv_reader:
        csv_data.append(row)

with open('FullTGV/DisEns-Rey100-Res' + str(res) + '-BGK.csv', 'r') as file:
    csv_reader = csv.reader(file, delimiter=';')
    for row in csv_reader:
        FullTGV_DisEns.append(row)

with open('FullTGV/DisEkin-Rey100-Res' + str(res) + '-BGK.csv', 'r') as file:
    csv_reader = csv.reader(file, delimiter=';')
    for row in csv_reader:
        FullTGV_DisEkin.append(row)

with open('FullTGV/DisSij-Rey100-Res' + str(res) + '-BGK.csv', 'r') as file:
    csv_reader = csv.reader(file, delimiter=';')
    for row in csv_reader:
        FullTGV_DisSij.append(row)

csv_data = np.array(csv_data, dtype=float)
FullTGV_DisEns= np.array(FullTGV_DisEns, dtype=float)
FullTGV_DisEkin= np.array(FullTGV_DisEkin, dtype=float)
FullTGV_DisSij= np.array(FullTGV_DisSij, dtype=float)

dissipation = np.array(simulation.reporters[1].out)

plt.figure(1)
plt.scatter(csv_data[:, 0], csv_data[:, 1], color='red', label='Brachet')
plt.plot(energy[:,1],-1/(np.pi*2)**3*np.gradient(64*energy[:,2],energy[:,1]),color = 'blue', label='Reduced')
plt.plot(FullTGV_DisEkin[:,0],FullTGV_DisEkin[:,1],color='green', label='Full')
plt.xlim(0,10)
plt.ylim(0,0.02)
plt.legend()
title = 'Dissipation trough kinetic energy'
plt.title(title)
plt.savefig(title)

plt.figure(2)
plt.scatter(csv_data[:, 0], csv_data[:, 1], color='red', label='Brachet')
plt.plot(dissipation[5:,1],1/(0.5*np.pi)**3*dissipation[5:,3], color = 'blue', label='Reduced')
plt.plot(FullTGV_DisEns[:,0],FullTGV_DisEns[:,1],color='green', label= 'Full')
plt.xlim(0,10)
plt.ylim(0,0.02)
plt.legend()
plt.title('Dissipation trough enstrophy')

plt.figure(3)
plt.scatter(csv_data[:, 0], csv_data[:, 1], color='red', label='Brachet')
plt.plot(dissipation[3:,1],dissipation[3:,2], color = 'blue',label='Reduced')
plt.plot(FullTGV_DisSij[:,0],FullTGV_DisSij[:,1],color = 'green',label='Full')
plt.legend()
plt.xlim(0,10)
plt.ylim(0,0.02)
plt.title('Dissipation trough shear moments')

plt.show()

(1401, 3)


FileNotFoundError: [Errno 2] No such file or directory: 'FullTGV/DisEns-Rey100-Res64-BGK.csv'

In [ ]:
u = flow.units.convert_velocity_to_pu(lattice.u(simulation.f)).numpy()
u_norm = np.linalg.norm(u,axis=0)
plt.imshow(u_norm)
plt.show()